- [previous file - EDA](2022-03-31_train-test_EDA.ipynb)
- [next tile - models with pre-tuned parameters and ensembles](2022-04-15_ensemble.ipynb)

## imports

In [4]:
from warnings import filterwarnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_percentage_error

from lib.model_related import *

sns.set()
filterwarnings("ignore")

## reading data

In [5]:
train_raw = pd.read_parquet("data/2022-04-08_train_pre-model.parquet")
test_raw = pd.read_parquet("data/2022-04-08_test_pre-model.parquet")

train_raw.shape, test_raw.shape

((115367, 30), (34686, 28))

In [6]:
def submit(hold_out: pd.DataFrame, model, name="submission"):
    preds = model.predict(hold_out)
    submission = pd.read_csv("data/sample_submission.csv")
    submission["price"] = preds
    submission.to_csv(f"{name}.csv", index=False)
    
    
def submit_log(hold_out: pd.DataFrame, model, name="submission"):
    preds = model.predict(hold_out)
    submission = pd.read_csv("data/sample_submission.csv")
    submission["price"] = np.exp(preds)
    submission.to_csv(f"{name}.csv", index=False)

In [7]:
train_raw["price"].sum()

158409758714.0

## encoding

In [7]:
train_raw["train/test"] = "train"
test_raw["train/test"] = "test"

data = train_raw.append(test_raw)
data["ptc"].fillna("Оригинал", inplace=True)

data[data.select_dtypes("object").columns.tolist()] = data[
    data.select_dtypes("object").columns.tolist()
].astype(str)

for col in set(data.select_dtypes(exclude=("object")).columns) - {"price"}:
    data[col] = (
        RobustScaler().fit_transform(data[col].values.reshape(-1, 1)).reshape(-1, 1)
    )

for col in ["model_name"]:
    data[col] = LabelEncoder().fit_transform(data[col].astype("str"))

data = pd.get_dummies(
    data,
    columns=[
        "vehicle_transmission",
        "vendor",
        "brand",
        "fuel_type",
        "body_type",
        "color",
        "ptc",
        "drive",
        "wheel",
        "age_cat",
    ],
)

train = data.loc[data["train/test"] == "train"]

train_jane = train.loc[train["sample"] == "jane"]
train_sokolov = train.loc[train["sample"] == "sokolov"]
train_jane["price"] = train_jane["price"] * 0.86
train = train_jane.append(train_sokolov)

train.drop(columns=["sample", "description", "train/test"], inplace=True)
test = data.loc[data["train/test"] == "test"].drop(
    columns=["sample", "description", "train/test", "price"]
)


## modelling

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop(columns="price"), train["price"], random_state = 42, shuffle=True)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((86525, 112), (86525,), (28842, 112), (28842,))

## model tuning

### RandomForest

In [12]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 14),
        'max_samples': trial.suggest_uniform('max_samples', 0.6, 0.99),
        'max_features': trial.suggest_categorical("max_features", ["auto", "sqrt", 'log2']),
        'max_depth': None,
        'bootstrap': True,
        'random_state': 42
    }

    rfr_o = RandomForestRegressor(**param)
    cv_roc_auc = cross_val_score(rfr_o, X_train, y_train, cv=3, scoring="neg_mean_absolute_percentage_error", n_jobs=-1)

    return np.mean(cv_roc_auc)


study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///RFRRegressor.db",
    study_name="RFRRegressor",
    load_if_exists=True,
)
study.optimize(objective, n_trials=200)

[I 2022-04-13 18:46:09,425] Using an existing study with name 'RFRRegressor' instead of creating a new one.
[I 2022-04-13 18:46:22,263] Trial 7 finished with value: -0.2503358825689884 and parameters: {'n_estimators': 209, 'min_samples_split': 12, 'min_samples_leaf': 7, 'max_samples': 0.6170731668968928, 'max_features': 'log2'}. Best is trial 3 with value: -0.15965928060336795.
[I 2022-04-13 18:46:41,167] Trial 8 finished with value: -0.20956902989733972 and parameters: {'n_estimators': 239, 'min_samples_split': 8, 'min_samples_leaf': 11, 'max_samples': 0.6960737412613612, 'max_features': 'sqrt'}. Best is trial 3 with value: -0.15965928060336795.
[I 2022-04-13 18:47:20,333] Trial 9 finished with value: -0.18754078091628576 and parameters: {'n_estimators': 472, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_samples': 0.9109759310947507, 'max_features': 'log2'}. Best is trial 3 with value: -0.15965928060336795.
[I 2022-04-13 18:47:39,851] Trial 10 finished with value: -0.20136877770

#### Version 1 (tuned by hand)

In [25]:
rf_tuned = RandomForestRegressor(
    random_state=42,
    n_estimators=800,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='log2',
    max_depth=None,
    bootstrap=True
).fit(X_train, np.log(y_train))

print("rf_tuned_log", mean_absolute_percentage_error(y_valid, np.exp(rf_tuned.predict(X_valid))))
submit_log(test, rf_tuned, "rf_tuned_log")
# rf_tuned_log 0.1315354887566232

#### Version 2

In [28]:
rf_optuned_174 = RandomForestRegressor(
    **{
        'n_estimators': 450, 
        'min_samples_split': 4, 
        'min_samples_leaf': 2, 
        'max_samples': 0.9899165552020569, 
        'max_features': 'auto',
        'random_state': 42,
        'max_depth': None,
        'bootstrap': True
    }
).fit(X_train, np.log(y_train))

print("rf_optuned_174_log", mean_absolute_percentage_error(y_valid, np.exp(rf_optuned_174.predict(X_valid))))
submit_log(test, rf_optuned_174, "rf_optuned_log_174")

rf_optuned_174_log 0.12683767196612786
